In [1]:
# -----------------------------
# Step 1: Install dependencies
# -----------------------------
# !pip install nltk spacy pyresparser scikit-learn pandas
!pip install pyresparser
!apt-get install -y poppler-utils
!python -m nltk.downloader all
!python -m spacy download en_core_web_sm


# Download NLTK and spaCy resources
# !python -m spacy download en_core_web_sm
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.7).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# -----------------------------
# Step 2: Resume Parsing
# -----------------------------
from pyresparser import ResumeParser

resume_path = '/content/Resume.pdf'

data = ResumeParser(resume_path).get_extracted_data()
print("Extracted Resume Data:")
print(data)


/usr/local/lib/python3.11/dist-packages/spacy/util.py:918: UserWarning: [W094] Model 'en_training' (0.0.0) specifies an under-constrained spaCy version requirement: >=2.1.4. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.8.5,<3.9.0
  warnings.warn(warn_msg)


OSError: [E053] Could not read config file from /usr/local/lib/python3.11/dist-packages/pyresparser/config.cfg

In [ ]:
# -----------------------------
# Step 3: Sample Job Dataset
# -----------------------------
import pandas as pd

# Example job descriptions (in real case, load from DB or CSV)
job_data = [
    {
        "job_title": "Data Scientist",
        "description": "Looking for a data scientist skilled in Python, machine learning, and statistics.",
        "skills_required": "Python, Machine Learning, Statistics"
    },
    {
        "job_title": "Software Engineer",
        "description": "Developer needed with strong Java and system design experience.",
        "skills_required": "Java, System Design, OOP"
    },
    {
        "job_title": "ML Engineer",
        "description": "We require an ML Engineer with experience in TensorFlow and deployment.",
        "skills_required": "TensorFlow, Deployment, Docker"
    },
    {
        "job_title": "Data Analyst",
        "description": "Role involves SQL, data visualization, and business intelligence tools.",
        "skills_required": "SQL, Tableau, Excel"
    }
]

df_jobs = pd.DataFrame(job_data)

In [ ]:
# -----------------------------
# Step 4: Text Preprocessing
# -----------------------------
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Combine job description and skills into one field
df_jobs['combined_text'] = df_jobs['description'] + ' ' + df_jobs['skills_required']

# Combine user's skills and other fields
user_text = ''
if data.get('skills'):
    user_text += ' '.join(data['skills'])
if data.get('experience'):
    user_text += ' ' + str(data['experience'])

In [ ]:
# -----------------------------
# Step 5: Compute Similarities
# -----------------------------
all_texts = [user_text] + list(df_jobs['combined_text'].values)

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(all_texts)

# Cosine similarity: user (index 0) vs all job descriptions
cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])[0]

# Add similarity scores to DataFrame
df_jobs['match_score'] = cosine_sim

# Sort by match score
recommended_jobs = df_jobs.sort_values(by='match_score', ascending=False)

In [ ]:
# -----------------------------
# Step 6: Show Recommendations
# -----------------------------
print("\nTop Job Recommendations:")
print(recommended_jobs[['job_title', 'match_score']])